## Importacion de librerias

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import snowflake.connector
from sqlalchemy import create_engine
from sklearn.preprocessing import LabelEncoder
import config 

### Lectura de la base

In [3]:
connection_url = f"snowflake://{config.user}:{config.password}@{config.account}/{config.database}/{config.schema}?warehouse={config.warehouse}"

# Crear el motor de conexión
engine = create_engine(connection_url)

# Verificar la conexión
try:
    with engine.connect() as connection:
        print("Conexión exitosa a Snowflake!")
except Exception as e:
    print(f"Error en la conexión: {e}")

Conexión exitosa a Snowflake!


In [4]:
raw = pd.read_sql("SELECT * FROM RAW.BANCO", engine)

In [5]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51000 entries, 0 to 50999
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   transaction_id                    51000 non-null  object 
 1   user_id                           51000 non-null  int64  
 2   transaction_amount                48480 non-null  float64
 3   transaction_type                  51000 non-null  object 
 4   time_of_transaction               48448 non-null  float64
 5   device_used                       48527 non-null  object 
 6   location                          48453 non-null  object 
 7   previous_fraudulent_transactions  51000 non-null  int64  
 8   account_age                       51000 non-null  int64  
 9   number_of_transactions_last_24h   51000 non-null  int64  
 10  payment_method                    48531 non-null  object 
 11  fraudulent                        51000 non-null  int64  
dtypes: f

Se identifica los valores nulos para decidir cómo deben ser tratados

In [6]:
print('Valores nulos:\n', raw.isnull().sum())

Valores nulos:
 transaction_id                         0
user_id                                0
transaction_amount                  2520
transaction_type                       0
time_of_transaction                 2552
device_used                         2473
location                            2547
previous_fraudulent_transactions       0
account_age                            0
number_of_transactions_last_24h        0
payment_method                      2469
fraudulent                             0
dtype: int64


In [ ]:
# Se crea una copia del dataframe original para evitar modificarlo directamente
curated = raw.copy()

In [ ]:
# Para las variables categoricas, se reemplazan los valores nulos por 'Unknown'
for col in curated.columns:
    if curated[col].dtype == 'object':  
        curated[col] = curated[col].fillna('Unknown') 

In [ ]:
# Para las variables numericas, se reemplazan los valores nulos por la media de la 'location' o 'device_used' dependiendo el contexto de lo que se quiere analizar
# Se reemplazan los valores nulos de 'transaction_amount' por la media de la 'location'
# Se reemplazan los valores nulos de 'time_of_transaction' por la media de la 'device_used'
def valores_nulos_num(curated):
    
    curated['transaction_amount'] = curated.groupby('location')['transaction_amount'].transform(lambda x: x.fillna(x.mean()))
    curated['time_of_transaction'] = curated.groupby('device_used')['time_of_transaction'].transform(lambda x: x.fillna(x.mean()))
    
    return curated
    
curated = valores_nulos_num(curated)

Se tiene un dataset curado, sin valores nulos ni duplicados

In [11]:
print('Valores nulos:\n', curated.isnull().sum())

Valores nulos:
 transaction_id                      0
user_id                             0
transaction_amount                  0
transaction_type                    0
time_of_transaction                 0
device_used                         0
location                            0
previous_fraudulent_transactions    0
account_age                         0
number_of_transactions_last_24h     0
payment_method                      0
fraudulent                          0
dtype: int64


Se exporta a la carpeta curated donde se almacena la base ya limpia y lista para realizar feature engineering

In [12]:
curated.to_csv('D:\Proyecto 2\Data\Clean\curated.csv', index=False)